In [1]:
from Ks import *
import pickle as pkl

# load data
f = open("data.pickle", "rb")
data = pkl.load(f)

In [2]:
import numpy as np

features = []
labels = []

for i in data:
    tmp = [i.bigCategory, i.day, i.goal, i.backers]
    features.append(tmp)
    labels.append(i.pleged)

X = np.array(features)
y = np.array(labels)

In [3]:
from sklearn.preprocessing import OneHotEncoder, normalize

# one hot encode category
category = X[:, [0]]
enc = OneHotEncoder(handle_unknown='ignore')
category = enc.fit_transform(category).toarray()

KeyboardInterrupt: 

In [4]:
# max normalize numeric features
d = X[:, [1,2,3]]
d = normalize(d, axis=0, norm="max")

In [5]:
X = np.concatenate((category, d), axis=1)

In [6]:
from sklearn.model_selection import KFold, train_test_split

# K-fold cross validation
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(X)

5

In [7]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

best_acc = 0

# use odd K to avoid tie
for i in range(11, 20, 2):

    for j, [train_index, test_index] in enumerate(kf.split(X)):
        X_train = X[train_index, :]
        y_train = y[train_index]

        X_test = X[test_index, :]
        y_test = y[test_index]

        # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print("Building on K={} fold={}".format(i, j))
        neigh = NearestNeighbors(n_neighbors=i)
        neigh.fit(X_train)

        print("Testing...")
        n = neigh.kneighbors(X_test, i, return_distance=False)
        pledge_predict = []
        state_predict = []
        actual_state = []

        for inx, nbrs in enumerate(n):
            nbrPlege = y_train[nbrs]
            p = np.mean(nbrPlege)
            pledge_predict.append(p)
            
            goal = X_test[inx][2]
            state = y_test[inx] > goal
            actual_state.append(state)

            # goal = X_test[inx][2]
            # state = X_test[inx][2] >= y_test[inx]
            # actual_state.append(state)

            if p >= goal:
                state_predict.append(True)
            else:
                state_predict.append(False)
        
        RMSE = np.sqrt((np.square(pledge_predict - y_test).mean(axis=0)))
        tn, fp, fn, tp = confusion_matrix(state_predict, actual_state).ravel()
        acc = accuracy_score(actual_state, state_predict)
        
        print("           Positive       Negative")
        print("Positive     {}             {}".format(tn, fp))
        print("Negative     {}             {}".format(fn, tp))

        print(classification_report(state_predict, actual_state))
        print("Accuracy: {}".format(acc))
        print("RMSE: {}".format(RMSE))

        if acc > best_acc:
            # save best classifier
            clf = open("KNN_clf.pkl", "wb")
            pkl.dump(neigh, clf)
            clf.close()

Building on K=11 fold=0
Testing...
           Positive       Negative
Positive     0             283
Negative     94828             14105
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       283
        True       0.98      0.13      0.23    108933

    accuracy                           0.13    109216
   macro avg       0.49      0.06      0.11    109216
weighted avg       0.98      0.13      0.23    109216

Accuracy: 0.12914774392030473
RMSE: 46239.26548907049
Building on K=11 fold=1
Testing...
           Positive       Negative
Positive     0             261
Negative     94797             14158
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       261
        True       0.98      0.13      0.23    108955

    accuracy                           0.13    109216
   macro avg       0.49      0.06      0.11    109216
weighted avg       0.98      0.13      0.23    109216

Accuracy: 0.1296